In [18]:
import pandas as pd
import requests
from io import StringIO
import os
from dotenv import load_dotenv


In [19]:
load_dotenv()

True

In [20]:
myurl = os.environ["RESUMEN_CIERE_URL"] + "?" + os.environ["AZURE_DATALAKE_GENAI_TOKEN"]
context = requests.get(myurl).text
df_resumen = pd.read_json(StringIO(context))


In [21]:

myurl = os.environ["DETALLE_TAREAS_URL"] + "?" + os.environ["AZURE_DATALAKE_GENAI_TOKEN"]
df = pd.read_parquet(myurl)


In [22]:
df

,FECHA_CIERRE,CODIGO_TAREA,DESCRIPCION_TAREA,INICIO,FIN,DURACION_SEGUNDOS,DURACION
0,2024-10-21,PX999C17,Depuracion del FSC017,2024-10-22 03:02:22,2024-10-22 03:02:23,1,00:00:01
1,2024-10-25,PX999C17,Depuracion del FSC017,2024-10-26 03:08:21,2024-10-26 03:08:22,1,00:00:01
2,2024-10-26,PX999C17,Depuracion del FSC017,2024-10-27 01:32:29,2024-10-27 01:32:30,1,00:00:01
3,2024-10-29,PX999C17,Depuracion del FSC017,2024-10-30 02:55:58,2024-10-30 02:55:59,1,00:00:01
4,2024-10-30,PX999C17,Depuracion del FSC017,2024-10-31 03:07:58,2024-10-31 03:07:59,1,00:00:01
...,...,...,...,...,...,...,...
2927,2024-11-13,PCSMI001B,Rendicion Batch,2024-11-14 03:18:44,2024-11-14 03:18:45,1,00:00:01
2928,2024-11-14,PCSMI001B,Rendicion Batch,2024-11-15 02:38:05,2024-11-15 02:38:06,1,00:00:01
2929,2024-11-15,PCSMI001B,Rendicion Batch,2024-11-16 02:36:57,2024-11-16 02:36:58,1,00:00:01
2930,2024-11-16,PCSMI001B,Rendicion Batch,2024-11-17 01:40:20,2024-11-17 01:40:21,1,00:00:01


In [31]:
df.groupby(["CODIGO_TAREA","DESCRIPCION_TAREA"], as_index=False,)["DURACION_SEGUNDOS"].agg(["max","min","mean","median","count"]	).sort_values("max",ascending=False).head(30)

,CODIGO_TAREA,DESCRIPCION_TAREA,max,min,mean,median,count
7,PAUSA,Pausa cargar interfaz SIAD new,7293,101,4049.652174,5077.0,23
5,PAUSA,PAUSA para cargar pagos,3810,31,832.391304,452.0,23
116,PP020FV2,PASAJE A HISTORICOS DE MOVIMI,3480,44,394.913043,46.0,23
6,PAUSA,PAUSA validar pagos procesados,2287,90,478.043478,156.0,23
113,PP020A,Pasaje a Historico FSH015-16 *,2219,209,433.217391,258.0,23
105,PNU00117,Pasaje Resultados FVC,1951,40,205.521739,50.0,23
106,PNU00217,Pasaje Resultados M/E a M/N Ac,1917,574,1132.260870,1073.0,23
152,PSNG917,Tablon de Cuotas,1877,356,454.000000,386.0,23
17,PBCGD017,GENERACION FSH012 NUEVO,1872,1667,1783.043478,1793.0,23
112,PP017,Depurador,1655,693,862.130435,808.0,23
